# 한글 깨짐 방지 (코랩)

In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (7,257 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 124935 files and dire

# Settings

In [1]:
# basic
import time
import random
import math

#data analytics
import numpy as np
import pandas as pd

#Math
import scipy as sp
import statsmodels.api as sm

#web crawling
import requests
from bs4 import BeautifulSoup

#visualization
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

# 브라우저에서 바로 그려지도록
%matplotlib inline

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')

# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

# 모델 유사도 평가 테스트

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

# 문장 예시
preds = [
    "안전관리교육철저에 대한 재발 방지 대책과 향후 조치 계획.",
    "안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획."
]

gts = [
    "안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획.",
    "안전교육 실시를 통한 재발 방지 대책과 향후 조치 계획."
]

# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

sample_scores = []

for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 768 Embedding Vector로 변환
    pred_embed = model.encode(pred)
    gt_embed = model.encode(gt)

    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

/usr/local/lib/python3.11/dist-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


예측 :  안전관리교육철저에 대한 재발 방지 대책과 향후 조치 계획.
정답 :  안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획.
Cosine Similarity Score :  0.82566565
--------------------
예측 :  안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.
정답 :  안전교육 실시를 통한 재발 방지 대책과 향후 조치 계획.
Cosine Similarity Score :  0.99174094
--------------------
전체 샘플의 Cosine Similarity Score 평균 :  0.9087033


# 모델 타겟칼럼 군집화 시도

## Data Load

In [13]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

train.head(3)

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,물적사고,공종,사고객체,작업프로세스,장소,부위,사고원인,재발방지대책 및 향후조치계획
0,TRAIN_00000,2023-12-31 오후 12:44,정규작업 -,맑음,1℃,30%,건축 / 건축물 / 근린생활시설,"4,892.77㎡","지상 14층, 지하 3층",떨어짐(5미터 이상 ~ 10미터 미만),없음,건축 > 철근콘크리트공사,건설자재 > 철근,설치작업,근린생활시설 / 내부,철근 / 고소,"고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
1,TRAIN_00001,2023-12-30 오후 03:35,정규작업 -,맑음,10℃,90%,토목 / 터널 / 철도터널,-,-,끼임,없음,토목 > 터널공사,건설자재 > 볼트,운반작업,철도터널 / 내부,볼트 / 바닥,부주의,재발 방지 대책 마련과 안전교육 실시.
2,TRAIN_00002,2023-12-30 오후 02:30,정규작업 -,맑음,14℃,70%,건축 / 건축물 / 업무시설,"1,994.62㎡","지상 5층, 지하 0층",넘어짐(미끄러짐),없음,건축 > 철근콘크리트공사,기타 > 기타,형틀 및 목공,업무시설 / 내부,기타 / 바닥,3층 슬라브 작업시 이동중 미끄러짐,현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조...


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns

def categorizing_sentences(sentences, num_of_clusters=3):
  """
  input :
  sentences(list) : 카테고라이징 하고 싶은 문장

  output :
  embeds (array) : 임베딩화된 각 문장
  labels (array) : 각 문장의 클러스터 번호
  """

  # 모델 로드 (대회 평가 모델)
  model = SentenceTransformer('jhgan/ko-sbert-sts')

  # 문장 임베딩
  embeddings = model.encode(sentences)

  # PCA로 2차원 축소 (시각화 용도)
  pca = PCA(n_components=2)
  embeddings_2d = pca.fit_transform(embeddings)

  # K-means 클러스터링 (클러스터 개수 k는 3으로 설정)
  num_clusters = num_of_clusters
  kmeans = KMeans(n_clusters=num_clusters, random_state=42)
  labels = kmeans.fit_predict(embeddings)

  # 시각화
  plt.figure(figsize=(8,6))
  sns.scatterplot(x=embeddings_2d[:,0], y=embeddings_2d[:,1], hue=labels, palette='viridis', s=100)
  for i, txt in enumerate(sentences):
      plt.annotate(txt[:10], (embeddings_2d[i,0], embeddings_2d[i,1]), fontsize=9)
  plt.title("Cluster Visualization of Safety Measures")
  plt.show()

  # 각 클러스터별 문장 출력
  for cluster_id in range(num_clusters):
      print(f"\n📌 클러스터 {cluster_id + 1}:")
      for i, sentence in enumerate(sentences):
          if labels[i] == cluster_id:
              print(f" - {sentence}")

  return embeddings, labels

In [ ]:
# 예제 문장들
# sentences = [
#     "고소작업 시 추락 위험이 있는 부위에 안전장비 설치.",
#     "재발 방지 대책 마련과 안전교육 실시.",
#     "현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조치 계획.",
#     "위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, 근로자 안전교육을 강화하며, 본 사고와 관련된 유사 피해 발생 방지를 위한 안전교육을 실시할 계획.",
#     "자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 후 장애물 사전 정리 작업 실시.",
#     "인근 작업장의 윈치양중 사고 강조 및 안전교육, 윈치양중 작업 최소화와 가능한 CAR작업 진행을 포함한 유사사고 재발 방지 대책 제출."
# ]

HYPERPARMS = {
    'num_of_clusters' : [3,5,7,9,11]
}

all_results = pd.DataFrame()
for num_of_cluster in HYPERPARMS['num_of_clusters']:
  result = pd.DataFrame()
  print('num_of_cluster : ', num_of_cluster)
  embeds, labels = categorizing_sentences(train['재발방지대책 및 향후조치계획'].values.tolist(), num_of_clusters=num_of_cluster)
  result['embedd'] = embeds.tolist()
  result['label'] = labels.tolist()
  result['target'] = train['target'].values.tolist()
  result['num_of_cluster'] = num_of_cluster
  all_results = pd.concat([all_results, result])

all_results.to_csv("all_target_embedding_clustering_results.csv",index=None,encoding='euc-kr')

In [ ]:
embeds, labels

In [ ]:
HYPERPARMS = {
    'num_of_clusters' : [3,5,7,9,11]
}

all_results = pd.DataFrame()
for num_of_cluster in HYPERPARMS['num_of_clusters']:
  result = pd.DataFrame()
  print('num_of_cluster : ', num_of_cluster)
  embeds, labels = categorizing_sentences(train['재발방지대책 및 향후조치계획'].values.tolist(), num_of_clusters=num_of_cluster)
  result['embedd'] = embeds.tolist()
  result['label'] = labels.tolist()
  result['target'] = train['target'].values.tolist()
  result['num_of_cluster'] = num_of_cluster
  all_results = pd.concat([all_results, result])

all_results.to_csv("all_target_embedding_clustering_results.csv",index=None,encoding='euc-kr')